<a href="https://colab.research.google.com/github/DaeSeokSong/image-processing/blob/feature%2FUnet-scar/RawDataset_Processor_Scar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RawDataset_Processor 실행 후 UNet 실행

# Import

In [1]:
# Image processing
import cv2
import numpy as np

from google.colab.patches import cv2_imshow
from google.colab import output

# ETC
import os
import time

## Mount google drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd /content/gdrive/MyDrive/Models/GAN_Scar
!ls -al

/content/gdrive/MyDrive/Models/GAN_Scar
total 159
drwx------ 2 root root  4096 Aug 16 10:50  Dataset
-rw------- 1 root root 86402 Aug 13 09:16  Image_segmentation-Scar.ipynb
drwx------ 4 root root  4096 Aug 16 08:39  Raw_Dataset
-rw------- 1 root root 11473 Aug 16 10:54  RawDataset_Processor-Scar.ipynb
-rw------- 1 root root 39995 Aug 15 11:40 'UNet architecture.PNG'
-rw------- 1 root root 15782 Aug 16 08:35  Unet-Scar.ipynb


# Grobal variable

In [4]:
# Path
MODEL_PATH = "/content/gdrive/MyDrive/Models/GAN_Scar"

RAW_TRAIN_SET_PATH = "/Raw_Dataset/train"
RAW_TEST_SET_PATH = "/Raw_Dataset/test"

DATASET_PATH = "/Dataset"

IMAGES_PATH = '/images'
LABELS_PATH = '/labels'

# Run

## Make dir

In [5]:
# Create processed dataset dir
train_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'train')
train_scars_dir = os.path.join(train_dir, 'scars')
train_labels_dir = os.path.join(train_dir, 'labels')

val_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'val')
val_scars_dir = os.path.join(val_dir, 'scars')
val_labels_dir = os.path.join(val_dir, 'labels')

test_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'test')
test_scars_dir = os.path.join(test_dir, 'scars')
test_labels_dir = os.path.join(test_dir, 'labels')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)

    if not os.path.exists(train_scars_dir):
        os.makedirs(train_scars_dir)
    if not os.path.exists(train_labels_dir):
        os.makedirs(train_labels_dir)

if not os.path.exists(val_dir):
    os.makedirs(val_dir)

    if not os.path.exists(val_scars_dir):
        os.makedirs(val_scars_dir)
    if not os.path.exists(val_labels_dir):
        os.makedirs(val_labels_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

    if not os.path.exists(test_scars_dir):
        os.makedirs(test_scars_dir)
    if not os.path.exists(test_labels_dir):
        os.makedirs(test_labels_dir)

## Process train dataset

In [6]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
os.chdir(image_path)
train_files = os.listdir(image_path)
train_files.sort()

# Divide train:val = 7:3
dataset_size = len(train_files)
train_size = int(dataset_size * 0.7)





# Create processed scar image for train
for idx in range(train_size): 
    image = cv2.imread(train_files[idx])

    # Get HSV image
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hue_image, sat_image, val_image = cv2.split(hsv_image)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(val_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])
    val_image = val_image[y:y+height, x:x+width]

    # Save normalized image
    # val_image는 흑백 이미지라 1채널에 각 픽셀값들도 0~255로 정규화 되어있다.
    cv2.imwrite(os.path.join(train_scars_dir, f'scar_{idx:03d}.png'), val_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    image = cv2.imread(train_files[idx])

    # Get HSV image
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hue_image, sat_image, val_image = cv2.split(hsv_image)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(val_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])
    val_image = val_image[y:y+height, x:x+width]

    # Save normalized image
    # val_image는 흑백 이미지라 1채널에 각 픽셀값들도 0~255로 정규화 되어있다.
    cv2.imwrite(os.path.join(val_scars_dir, f'scar_{val_idx:03d}.png'), val_image)





# Load raw label images
os.chdir(label_path)
label_files = os.listdir(label_path)
label_files.sort()

# Create processed scar label for train
for idx in range(train_size):
    label_image = cv2.imread(label_files[idx], cv2.IMREAD_GRAYSCALE)
    
    # Save normalized image
    # 그레이 스케일은 흑백으로, 1채널에 각 픽셀값들도 0~255로 정규화 되어있다.
    cv2.imwrite(os.path.join(train_labels_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar label for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    label_image = cv2.imread(label_files[idx], cv2.IMREAD_GRAYSCALE)
    
    # Save normalized image
    # 그레이 스케일은 흑백으로, 1채널에 각 픽셀값들도 0~255로 정규화 되어있다.
    cv2.imwrite(os.path.join(val_labels_dir, f'label_{val_idx:03d}.png'), label_image)

## Process test dataset

In [7]:
image_path = MODEL_PATH + RAW_TEST_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TEST_SET_PATH + LABELS_PATH





# Load raw train images
os.chdir(image_path)
train_files = os.listdir(image_path)
train_files.sort()

# Create processed scar image for train
for idx in range(len(train_files)): 
    image = cv2.imread(train_files[idx])

    # Get HSV image
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hue_image, sat_image, val_image = cv2.split(hsv_image)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(val_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])
    val_image = val_image[y:y+height, x:x+width]

    # Save normalized image
    # val_image는 흑백 이미지라 1채널에 각 픽셀값들도 0~255로 정규화 되어있다.
    cv2.imwrite(os.path.join(test_scars_dir, f'scar_{idx:03d}.png'), val_image)





# Load raw label images
os.chdir(label_path)
label_files = os.listdir(label_path)
label_files.sort()

# Create processed scar label for train
for idx in range(len(label_files)):
    label_image = cv2.imread(label_files[idx], cv2.IMREAD_GRAYSCALE)
    
    # Save normalized image
    # 그레이 스케일은 흑백으로, 1채널에 각 픽셀값들도 0~255로 정규화 되어있다.
    cv2.imwrite(os.path.join(test_labels_dir, f'label_{idx:03d}.png'), label_image)